In [1]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [2]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [3]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Sun Apr 16 04:52:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
%%writefile l9q1a.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
__global__ void add_a(int *a, int *b, int *t)
{
	int size=gridDim.x;
	int ind = blockIdx.x;
	int n=sizeof(a)/size;

	for(int i=0;i<n;i++)
		t[n*ind+i] = a[n*ind+i] + b[n*ind+i];
}
int main(void)
{
	int m,n,i,j;
	int *d_a,*d_t, *d_b;
	printf("Enter the value of m: ");scanf("%d",&m);
	printf("Enter the value of n: ");scanf("%d",&n);
	int size=sizeof(int)*m*n;

	printf("Enter input matrix A:\n");

	int a[m*n], b[m*n], t[m*n];
	for(i=0;i<m*n;i++)
		scanf("%d",&a[i]);

	printf("Enter input matrix B:\n");
	for(i=0;i<m*n;i++)
		scanf("%d",&b[i]);

	cudaMalloc((void**)&d_a,size);
	cudaMalloc((void**)&d_b,size);
	cudaMalloc((void**)&d_t,size);

	cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);
	cudaMemcpy(d_b,b,size,cudaMemcpyHostToDevice);

	dim3 bb(m,n);
	add_a<<<m, 1>>>(d_a, d_b, d_t);
	printf("cuda successfully ran!\n");

	cudaMemcpy(t,d_t,size,cudaMemcpyDeviceToHost);
	printf("cuda successfully ran!\n");

	printf("Result matrix is:\n");
	for(i=0;i<m;i++)
	{
		for(j=0;j<n;j++)
			printf("%d\t",t[i*m+j]);
		printf("\n");
	}

	getchar();
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_t);
	return 0;
}

Writing l9q1a.cu


In [9]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 l9q1a.cu -o l9q1a

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [10]:
!./l9q1a

Enter the value of m: 2
Enter the value of n: 2
Enter input matrix A:
1 2 3 4
Enter input matrix B:
1 2 3 4
cuda successfully ran!
cuda successfully ran!
Result matrix is:
2	4	
6	8	
